In [32]:
import sys
sys.path.append('../script/')
import os
from os.path import exists
from datetime import datetime
import json
import gc
from functools import partial
from collections import OrderedDict

import numpy as np
import pandas as pd
from sklearn.preprocessing import QuantileTransformer
from sklearn.decomposition import PCA
import torch
import torch.nn as nn
from torch.utils.tensorboard import SummaryWriter
# from adabelief_pytorch import AdaBelief

import utils
import models
import train as trainer
DEVICE = "cuda"
# DEVICE = "cpu"
EPOCHS = 3000
MODELNAME = "Baseline1125"
if not exists(MODELNAME):
    os.makedirs(f"{MODELNAME}/tensorboard")
now = datetime.now()
now = str(now)[5:17].replace(" ", "_").replace(":", "")
writer = SummaryWriter(log_dir=f"{MODELNAME}/tensorboard")

In [33]:
df = pd.read_csv("../input/folds/train.csv")
with open("../input/folds/targets", "r") as f:
    targets = f.read().split("\n")
with open("../input/folds/features", "r") as f:
    features = f.read().split("\n")
targets = targets[:-1]

## Quantile scaler + PCA

In [70]:
def quantilePCA(df_tr, df_va, drop=True, seed=42):
    gcols = [f'g-{i}' for i in range(772)]
    ccols = [f'c-{i}' for i in range(100)]
    
    gdf_tr, gdf_va = df_tr[gcols].copy(), df_va[gcols].copy()
    cdf_tr, cdf_va = df_tr[ccols].copy(), df_va[ccols].copy()    
    print('QT  ',end='')
    gqt = QuantileTransformer(random_state=seed)
    cqt = QuantileTransformer(random_state=seed)
    gdf_tr = gqt.fit_transform(gdf_tr)
    cdf_tr = cqt.fit_transform(cdf_tr)
    gdf_va = gqt.transform(gdf_va)
    cdf_va = cqt.transform(cdf_va)
    
    print('PCA ',end='')
    n_gpc, n_cpc = 80, 10
    gpc = PCA(n_components=n_gpc, random_state=seed)
    cpc = PCA(n_components=n_cpc, random_state=seed)
    gdf_tr = gpc.fit_transform(gdf_tr)
    cdf_tr = cpc.fit_transform(cdf_tr)
    gdf_va = gpc.transform(gdf_va)
    cdf_va = cpc.transform(cdf_va)
    gcols_pca = [f'g-qpc{i}' for i in range(n_gpc)]
    ccols_pca = [f'c-qpc{i}' for i in range(n_cpc)]
    df_tr.loc[:, gcols_pca], df_tr.loc[:, ccols_pca] = gdf_tr, cdf_tr
    df_va.loc[:, gcols_pca], df_va.loc[:, ccols_pca] = gdf_va, cdf_va
    
    
    if drop:
        print('DROP')
        cols = gcols + ccols
        print(df_tr.columns)
        print(df_va.columns)
        df_tr.drop(cols, axis=1, inplace=True)
        df_va.drop(cols, axis=1, inplace=True)
        cols_order = ['sig_id'] + gcols_pca + ccols_pca + targets + ['kfold'] + ['nsc_labels']
    else:
        cols_order = ['sig_id'] + gcols + ccols + gcols_pca + ccols_pca + targets + ['kfold'] + ['nsc_labels']
    df_tr, df_va = df_tr[cols_order], df_va[cols_order]
    return df_tr, df_va

In [72]:
df = pd.read_csv("../input/folds/train.csv")
display(df)
df_tr, df_va = quantilePCA(df.copy(), df.copy(), drop=True)
display(df_tr)

,sig_id,g-0,g-1,g-2,g-3,g-4,g-5,g-6,g-7,g-8,...,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor,kfold,nsc_labels
0,id_000644bb2,1.0620,0.5577,-0.2479,-0.6208,-0.1944,-1.0120,-1.0220,-0.0326,0.5548,...,0,0,0,0,0,0,0,0,4,0
1,id_000779bfc,0.0743,0.4087,0.2991,0.0604,1.0190,0.5207,0.2341,0.3372,-0.4047,...,0,0,0,0,0,0,0,0,2,0
2,id_000a6266a,0.6280,0.5817,1.5540,-0.0764,-0.0323,1.2390,0.1715,0.2155,0.0065,...,0,0,0,0,0,0,0,0,4,0
3,id_0015fd391,-0.5138,-0.2491,-0.2656,0.5288,4.0620,-0.8095,-1.9590,0.1792,-0.1321,...,0,0,0,0,0,0,0,0,0,0
4,id_001626bd3,-0.3254,-0.4009,0.9700,0.6919,1.4180,-0.8244,-0.2800,-0.1498,-0.8789,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23809,id_fffb1ceed,0.1394,-0.0636,-0.1112,-0.5080,-0.4713,0.7201,0.5773,0.3055,-0.4726,...,0,0,0,0,0,0,0,0,0,0
23810,id_fffb70c0c,-1.3260,0.3478,-0.3743,0.9905,-0.7178,0.6621,-0.2252,-0.5565,0.5112,...,0,0,0,0,0,0,0,0,4,0
23811,id_fffc1c3f4,0.3942,0.3756,0.3109,-0.7389,0.5505,-0.0159,-0.2541,0.1745,-0.0340,...,0,0,0,0,0,0,0,0,3,0
23812,id_fffcb9e7c,0.6660,0.2324,0.4392,0.2044,0.8531,-0.0343,0.0323,0.0463,0.4299,...,0,0,0,0,0,0,0,0,4,0


QT  PCA 

,sig_id,g-0,g-1,g-2,g-3,g-4,g-5,g-6,g-7,g-8,...,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor,kfold,nsc_labels
0,id_000644bb2,1.0620,0.5577,-0.2479,-0.6208,-0.1944,-1.0120,-1.0220,-0.0326,0.5548,...,0,0,0,0,0,0,0,0,4,0
1,id_000779bfc,0.0743,0.4087,0.2991,0.0604,1.0190,0.5207,0.2341,0.3372,-0.4047,...,0,0,0,0,0,0,0,0,2,0
2,id_000a6266a,0.6280,0.5817,1.5540,-0.0764,-0.0323,1.2390,0.1715,0.2155,0.0065,...,0,0,0,0,0,0,0,0,4,0
3,id_0015fd391,-0.5138,-0.2491,-0.2656,0.5288,4.0620,-0.8095,-1.9590,0.1792,-0.1321,...,0,0,0,0,0,0,0,0,0,0
4,id_001626bd3,-0.3254,-0.4009,0.9700,0.6919,1.4180,-0.8244,-0.2800,-0.1498,-0.8789,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23809,id_fffb1ceed,0.1394,-0.0636,-0.1112,-0.5080,-0.4713,0.7201,0.5773,0.3055,-0.4726,...,0,0,0,0,0,0,0,0,0,0
23810,id_fffb70c0c,-1.3260,0.3478,-0.3743,0.9905,-0.7178,0.6621,-0.2252,-0.5565,0.5112,...,0,0,0,0,0,0,0,0,4,0
23811,id_fffc1c3f4,0.3942,0.3756,0.3109,-0.7389,0.5505,-0.0159,-0.2541,0.1745,-0.0340,...,0,0,0,0,0,0,0,0,3,0
23812,id_fffcb9e7c,0.6660,0.2324,0.4392,0.2044,0.8531,-0.0343,0.0323,0.0463,0.4299,...,0,0,0,0,0,0,0,0,4,0


In [3]:
# TODO: クラスタごとに並べてConv1d
class BaseLine(nn.Module):
    def __init__(self, num_features, num_targets, num_layers=3, dropout=.2, hidden_size=256, activation="relu", batchnorm=True, weight_norm=True):
        super().__init__()
        layers = []
        for _ in range(num_layers):
            layers.append(nn.utils.weight_norm(nn.Linear(num_features if len(layers)==0 else hidden_size, hidden_size, bias=(not batchnorm))))
            if batchnorm:
                layers.append(nn.BatchNorm1d(hidden_size))
            layers.append(nn.Dropout(dropout))
            if activation == "relu":
                layers.append(nn.ReLU())
            elif activation == "prelu":
                layers.append(nn.PReLU())
            else:
                raise RuntimeError(f'{activation} is not implemented')
        # layers.append(nn.utils.weight_norm(nn.Linear(hidden_size, num_targets)))
        layers.append(nn.Linear(hidden_size, num_targets))
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        x = self.model(x)
        return x

In [4]:
def set_output_bias(model, df, targets):   
    init_bias = np.array([])
    for target in targets:
        try:
            neg, pos = np.bincount(df[target])
        except ValueError:
            neg, pos = np.array([df.shape[0], 0.01])
        init_bias_ = np.log([pos/neg])
        init_bias = np.append(init_bias, init_bias_)
    model.model[-1].bias.data = torch.tensor(init_bias, dtype=torch.float32)
    return model

In [5]:
class LabelSmoothingCrossEntropy(nn.Module):
    def __init__(self):
        super(LabelSmoothingCrossEntropy, self).__init__()
    def forward(self, x, target, smoothing=0.2):
        confidence = 1. - smoothing
        logprobs = F.log_softmax(x, dim=-1)
        nll_loss = -logprobs.gather(dim=-1, index=target.unsqueeze(1))
        nll_loss = nll_loss.squeeze(1)
        smooth_loss = -logprobs.mean(dim=-1)
        loss = confidence * nll_loss + smoothing * smooth_loss
        return loss.mean()
from utils import LabelSmoothingCrossEntropy

# criterion = LabelSmoothingCrossEntropy()
# loss = criterion(outputs, targets)
# loss.backward()
# optimizer.step()

In [6]:
def run_training(df, fold, params, hp_tune=False):

    save_model = False if hp_tune else True
    print(f'\n[Fold No.{fold:>3}]')
    train_df = df[df.kfold != fold].reset_index(drop=True)
    valid_df = df[df.kfold == fold].reset_index(drop=True)

    train_df, valid_df = quantilePCA(train_df, valid_df, drop=True, seed=42)

    x_tr = train_df[features].to_numpy()
    x_va = valid_df[features].to_numpy()

    y_tr = train_df[targets].to_numpy()
    y_va = valid_df[targets].to_numpy()

    dataset_tr = utils.MoaDataset(x_tr, y_tr)
    loader_tr = torch.utils.data.DataLoader(dataset_tr, batch_size=512, num_workers=2, pin_memory=True)
    dataset_va = utils.MoaDataset(x_va, y_va)
    loader_va = torch.utils.data.DataLoader(dataset_va, batch_size=512, num_workers=2, pin_memory=True)

    model = BaseLine(num_features=x_tr.shape[1], num_targets=y_tr.shape[1], **params['nn_params'])
    model = set_output_bias(model, train_df, targets)
    model.to(DEVICE)

    if params["optimizer"] == "SGD":
        optimizer = torch.optim.SGD(model.parameters(), **params["optim_params"])
    elif params["optimizer"] == "Adam":
        optimizer = torch.optim.Adam(model.parameters(), **params["optim_params"])
    elif params["optimizer"] == "AdamW":
        optimizer = torch.optim.AdamW(model.parameters(), **params["optim_params"])
    elif params["optimizer"] == "AdaBelief":
        optimizer = AdaBelief(model.parameters(), **params["optim_params"])
    else:
        raise RuntimeError(f'{params["optimizer"]} is not implemented')

    if params["scheduler"] == "ReduceLROnPlateau":
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="min", **params["scdl_params"])
    elif params["scheduler"] == "CosineAnnealingLR":
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, **params["scdl_params"])
    elif params["scheduler"] == "none": 
        print("No scheduling will be applied")
        scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda ep: 1**ep)
    else:
        raise RuntimeError(f'{params["scheduler"]} is not implemented')

    eng = utils.Engine(model, optimizer, device=DEVICE)

    del df, train_df, valid_df, x_tr, x_va, y_tr, y_va
    gc.collect()

    torch.backends.cudnn.benchmark = True

    print(f'Training state is shown in {MODELNAME}/tensorboard')
    filename = f"{MODELNAME}/{now}_fold{fold}.pt"

    loss_best = np.inf
    patience = 25
    patience_cnt = 0
    for ep in range(EPOCHS):
        loss_tr = eng.train(loader_tr)
        loss_tr_nodrop = eng.validate(loader_tr)
        loss_va = eng.validate(loader_va)
        scheduler.step(loss_va)
        print(f'Ep.{ep:>3}/{EPOCHS:>3}, patience:{patience_cnt:>2}/{patience:>2}, train:{loss_tr:.6}, tr_nodrop:{loss_tr_nodrop:.6}, valid:{loss_va:.6}', end='\r')
        writer.add_scalars(f'{now}/fold{fold}', {'train':loss_tr, 'tr_nodrop':loss_tr_nodrop, 'valid':loss_va}, ep)
        if loss_va < loss_best:
            patience_cnt = 0
            loss_best = loss_va
            if save_model:
                torch.save(model.model.state_dict(), filename)
        else:
            patience_cnt += 1
        if patience_cnt > patience:
            break

    print("\nmodel saved at:", filename)

In [7]:
params = {
    "nn_params": {"dropout": 0.5, "num_layers":4, "hidden_size": 512, "activation": "relu", "batchnorm": True, "weight_norm": True},
    "optimizer": "Adam",
    # # SGD
    # "optim_params": {"lr":1e-4, "momentum": 0.3, "weight_decay": 0.2, "dampening": 0, "nesterov": False},
    # Adam
    "optim_params": {"lr":1e-2, "betas": (0.9, 0.999), "eps": 1e-08, "weight_decay": 1.2e-6, "amsgrad": False},
    # # Adabelief 
    # "optim_params": {"lr": 1e-2, "eps":1e-16, "betas": (0.9,0.999), "weight_decay": 1.2e-6, "weight_decouple": False, "rectify": True, "fixed_decay": False, "amsgrad": False},
    "scheduler": "ReduceLROnPlateau",
    "scdl_params": {"threshold": 1e-5, "patience": 3}
    # # ReduceLROnPlateau
    # "scdl_params": {"threshold": 1e-5, "patience": 3}
    # # CosineAnnealingLR
    # "scdl_params": {"T_max":8, "eta_min":0, "last_epoch":-1}
}
# 0.02355, 0.03 on momentum:0

In [8]:
with open(f'{MODELNAME}/{now}_params.json', 'w') as f:
    json.dump(params, f, indent=4)

In [9]:
%%time
for fold in range(5):
    run_training(df, fold, params)


[Fold No.  0]
Training state is shown in Baseline1122/tensorboard
Ep. 47/3000, patience:25/25, train:0.0148142, tr_nodrop:0.0140421, valid:0.0164846
model saved at: Baseline1122/11-26_0045_fold0.pt

[Fold No.  1]
Training state is shown in Baseline1122/tensorboard
Ep. 43/3000, patience:25/25, train:0.0147418, tr_nodrop:0.0139523, valid:0.0160776
model saved at: Baseline1122/11-26_0045_fold1.pt

[Fold No.  2]
Training state is shown in Baseline1122/tensorboard
Ep. 45/3000, patience:25/25, train:0.0144253, tr_nodrop:0.0136617, valid:0.0164775
model saved at: Baseline1122/11-26_0045_fold2.pt

[Fold No.  3]
Training state is shown in Baseline1122/tensorboard
Ep. 51/3000, patience:25/25, train:0.0144511, tr_nodrop:0.0136164, valid:0.0159169
model saved at: Baseline1122/11-26_0045_fold3.pt

[Fold No.  4]
Training state is shown in Baseline1122/tensorboard
Ep. 48/3000, patience:25/25, train:0.0145504, tr_nodrop:0.0138211, valid:0.0165258
model saved at: Baseline1122/11-26_0045_fold4.pt
Wall 

In [17]:
writer.close()

In [18]:
gc.collect()

88

---
## Get CV Score

In [12]:
predictions = np.zeros((df.shape[0], len(targets)))
for fold in range(5):
    filename = f"{MODELNAME}/{now}_fold{fold}.pt"
    print(f'[Fold No.{fold:>3}] Predicting...', end='\r')
    train_df = df[df.kfold != fold].reset_index(drop=True)
    valid_df = df[df.kfold == fold].reset_index(drop=True)
    va_idx = df[df.kfold == fold].index
    
    x_tr = train_df[features].to_numpy()
    x_va = valid_df[features].to_numpy()

    y_tr = train_df[targets].to_numpy()
    y_va = valid_df[targets].to_numpy()

    dataset_tr = utils.MoaDataset(x_tr, y_tr)
    loader_tr = torch.utils.data.DataLoader(dataset_tr, batch_size=512, num_workers=2, pin_memory=True)
    dataset_va = utils.MoaDataset(x_va, y_va)
    loader_va = torch.utils.data.DataLoader(dataset_va, batch_size=512, num_workers=2, pin_memory=True)
    
    model = BaseLine(num_features=x_tr.shape[1], num_targets=y_tr.shape[1], **params['nn_params'])
    
    weight = torch.load(filename, map_location=torch.device(DEVICE))
    weight = OrderedDict([(f'model.{k}', v) for k, v in weight.items()])
    model.load_state_dict(weight)
    
    model.eval()
    ps = []
    for ind, batch in enumerate(loader_va):
        ps.append(torch.sigmoid(model(batch["x"])).detach().cpu().numpy())
    ps = np.vstack(ps)
    predictions[va_idx] += ps
print()

[Fold No.  4] Predicting...


In [13]:
def log_loss_metric(y_true, y_pred):
    y_pred_clip = np.clip(y_pred, 1e-15, 1 - 1e-15)
    loss = - np.mean(np.mean(y_true * np.log(y_pred_clip) + (1 - y_true) * np.log(1 - y_pred_clip), axis = 1))
    return loss
print(f'CV score               : {log_loss_metric(df[targets].values, predictions):.6}')

CV score               : 0.016269


In [14]:
oof = pd.read_csv('../input/lish-moa/train_targets_scored.csv')
oof.iloc[:, 1:] = predictions
oof.to_csv(f"{MODELNAME}/oof.csv")

In [15]:
predictions_ = predictions.copy()
predictions_ = np.clip(predictions_,0.0005,0.999)
predictions_[df["cp_type_ctl_vehicle"]==1] = 0
print(f'CV score w/ postprocess: {log_loss_metric(df[targets].values, predictions_):.6}')

CV score w/ postprocess: 0.0161582


In [16]:

print(f'CV score w/ postprocess: {log_loss_metric(df[targets].values, predictions_):.6}')

CV score w/ postprocess: 0.0161582


In [2]:
# TODO: クラスタごとに並べてConv1d
class BaseLine(nn.Module):
    def __init__(self, num_features, num_targets, num_layers=3, dropout=.2, hidden_size=256, activation="relu", batchnorm=True, weight_norm=True):
        super().__init__()
        layers = []
        for _ in range(num_layers):
            layers.append(nn.utils.weight_norm(nn.Linear(num_features if len(layers)==0 else hidden_size, hidden_size, bias=(not batchnorm))))
            if batchnorm:
                layers.append(nn.BatchNorm1d(hidden_size))
            layers.append(nn.Dropout(dropout))
            if activation == "relu":
                layers.append(nn.ReLU())
            elif activation == "prelu":
                layers.append(nn.PReLU())
            else:
                raise RuntimeError(f'{activation} is not implemented')
        # layers.append(nn.utils.weight_norm(nn.Linear(hidden_size, num_targets)))
        layers.append(nn.Linear(hidden_size, num_targets))
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        x = self.model(x)
        return x

In [3]:
df = pd.read_csv("../input/folds/train.csv")
with open("../input/folds/targets", "r") as f:
    targets = f.read().split("\n")[:-1]
with open("../input/folds/features", "r") as f:
    features = f.read().split("\n")
with open(PARAMS_PATH) as f:
    params = json.load(f)

In [4]:
def log_loss_metric(y_true, y_pred):
    y_pred_clip = np.clip(y_pred, 1e-15, 1 - 1e-15)
    loss = - np.mean(np.mean(y_true * np.log(y_pred_clip) + (1 - y_true) * np.log(1 - y_pred_clip), axis = 1))
    return loss

---
## Predicting as usual

In [5]:
predictions = np.zeros((df.shape[0], len(targets)))
for fold in range(5):
    print(f'[Fold No.{fold:>3}] Predicting...', end='\r')
    train_df = df[df.kfold != fold].reset_index(drop=True)
    valid_df = df[df.kfold == fold].reset_index(drop=True)
    va_idx = df[df.kfold == fold].index
    
    x_tr = train_df[features].to_numpy()
    x_va = valid_df[features].to_numpy()

    y_tr = train_df[targets].to_numpy()
    y_va = valid_df[targets].to_numpy()

    dataset_tr = utils.MoaDataset(x_tr, y_tr)
    loader_tr = torch.utils.data.DataLoader(dataset_tr, batch_size=512, num_workers=2, pin_memory=True)
    dataset_va = utils.MoaDataset(x_va, y_va)
    loader_va = torch.utils.data.DataLoader(dataset_va, batch_size=512, num_workers=2, pin_memory=True)
    
    model = BaseLine(num_features=x_tr.shape[1], num_targets=y_tr.shape[1], **params['nn_params'])
    
    weight = torch.load(WEIGHT_PATHS[fold], map_location=torch.device(DEVICE))
    weight = OrderedDict([(f'model.{k}', v) for k, v in weight.items()])
    model.load_state_dict(weight)
    
    model.eval()
    ps = []
    for ind, batch in enumerate(loader_va):
        ps.append(torch.sigmoid(model(batch["x"])).detach().cpu().numpy())
    ps = np.vstack(ps)
    predictions[va_idx] += ps
print()

[Fold No.  4] Predicting...


In [6]:
print(f'CV score               : {log_loss_metric(df[targets].values, predictions):.6}')
predictions_ = predictions.copy()
predictions_ = np.clip(predictions_,0.0005,0.999)
predictions_[df["cp_type_ctl_vehicle"]==1] = 0
print(f'CV score w/ postprocess: {log_loss_metric(df[targets].values, predictions_):.6}')

CV score               : 0.016269
CV score w/ postprocess: 0.0161582


## Predicting w/ dropout ensemble

In [ ]:
sg = nn.Sigmoid()
N_TRIALS = 100
predictions = np.zeros((df.shape[0], len(targets)))
for seed in range(N_TRIALS):
    for fold in range(5):
        print(f'[TRIAL No. {seed:>3}] [Fold No.{fold:>3}] Predicting...', end='\r')
        train_df = df[df.kfold != fold].reset_index(drop=True)
        valid_df = df[df.kfold == fold].reset_index(drop=True)
        va_idx = df[df.kfold == fold].index

        x_tr = train_df[features].to_numpy()
        x_va = valid_df[features].to_numpy()

        y_tr = train_df[targets].to_numpy()
        y_va = valid_df[targets].to_numpy()

        dataset_tr = utils.MoaDataset(x_tr, y_tr)
        loader_tr = torch.utils.data.DataLoader(dataset_tr, batch_size=512, num_workers=2, pin_memory=True)
        dataset_va = utils.MoaDataset(x_va, y_va)
        loader_va = torch.utils.data.DataLoader(dataset_va, batch_size=512, num_workers=2, pin_memory=True)

        model = BaseLine(num_features=x_tr.shape[1], num_targets=y_tr.shape[1], **params['nn_params'])

        weight = torch.load(WEIGHT_PATHS[fold], map_location=torch.device(DEVICE))
        weight = OrderedDict([(f'model.{k}', v) for k, v in weight.items()])
        model.load_state_dict(weight)

        model.train()
        ps = []
        for ind, batch in enumerate(loader_va):
            ps.append(torch.sigmoid(model(batch["x"])).detach().cpu().numpy())
        ps = np.vstack(ps)
        predictions[va_idx] += ps

[TRIAL No.   0] [Fold No.  4] Predicting...
[TRIAL No.   1] [Fold No.  4] Predicting...
[TRIAL No.   2] [Fold No.  4] Predicting...
[TRIAL No.   3] [Fold No.  4] Predicting...
[TRIAL No.   4] [Fold No.  4] Predicting...
[TRIAL No.   5] [Fold No.  4] Predicting...
[TRIAL No.   6] [Fold No.  4] Predicting...
[TRIAL No.   7] [Fold No.  4] Predicting...
[TRIAL No.   8] [Fold No.  4] Predicting...
[TRIAL No.   9] [Fold No.  4] Predicting...
[TRIAL No.  10] [Fold No.  4] Predicting...
[TRIAL No.  11] [Fold No.  4] Predicting...
[TRIAL No.  12] [Fold No.  4] Predicting...
[TRIAL No.  13] [Fold No.  4] Predicting...
[TRIAL No.  14] [Fold No.  4] Predicting...
[TRIAL No.  15] [Fold No.  4] Predicting...
[TRIAL No.  16] [Fold No.  4] Predicting...
[TRIAL No.  17] [Fold No.  4] Predicting...
[TRIAL No.  18] [Fold No.  4] Predicting...
[TRIAL No.  19] [Fold No.  4] Predicting...
[TRIAL No.  20] [Fold No.  4] Predicting...
[TRIAL No.  21] [Fold No.  4] Predicting...
[TRIAL No.  22] [Fold No.  4] Pr

In [8]:
print(f'CV score               : {log_loss_metric(df[targets].values, predictions):.6}')
predictions_ = predictions.copy()
predictions_ = np.clip(predictions_,0.0005,0.999)
predictions_[df["cp_type_ctl_vehicle"]==1] = 0
print(f'CV score w/ postprocess: {log_loss_metric(df[targets].values, predictions_):.6}')

CV score               : 0.0163489
CV score w/ postprocess: 0.0162451
